In [1]:
# from https://tryolabs.com/blog/2015/02/17/python-elasticsearch-first-steps/

# make sure ES is up and running
import requests
#res = requests.get('http://localhost:9200')
res = requests.get('https://search-cbsnews-aswan-es-kdnk6rytsth2a2s3o53qhw4huy.us-east-1.es.amazonaws.com')
print(res.content)

b'{\n  "name" : "PzcO3EK",\n  "cluster_name" : "839089019834:cbsnews-aswan-es",\n  "cluster_uuid" : "a4WuC2x3S-ClnvBCrvnKoQ",\n  "version" : {\n    "number" : "5.5.2",\n    "build_hash" : "Unknown",\n    "build_date" : "2017-10-18T04:35:01.381Z",\n    "build_snapshot" : false,\n    "lucene_version" : "6.6.0"\n  },\n  "tagline" : "You Know, for Search"\n}\n'


In [35]:
#connect to our cluster
from elasticsearch import Elasticsearch
#es = Elasticsearch([{'host': 'localhost', 'port': 9200}])
es = Elasticsearch([{'host': 'search-cbsnews-aswan-es-kdnk6rytsth2a2s3o53qhw4huy.us-east-1.es.amazonaws.com', 'port': 80}])

In [29]:
import csv

with open('/Users/chada/Desktop/Archive/Programming/Node/geneology/data/Geneology.csv') as csv_file:
    people = list(csv.DictReader(csv_file))

In [12]:
people[0]

{'': '',
 'Born': '',
 'Born Location': '',
 'Died': '',
 'Died Location': '',
 'Father': '',
 'Gender': 'Male',
 'Married Date': '',
 'Married Location': '',
 'Married to': 'Ann Waldron',
 'Mother': '',
 'Name': 'Joseph Armstrong',
 'Notes': '',
 'Second Marriage Date': '',
 'Second Marriage Location': '',
 'Second Marriage to': '',
 'Third Marriage Date': '',
 'Third Marriage to': ''}

In [30]:
import json 

i = 1
for person in people[1:6]:
    #print(person)    
    born_date = person['Born']
    died_date = person['Died']
    married_date = person['Married Date']

    if len(born_date) == 4:
        born_date = '01/01/' + born_date
    if len(died_date) == 4:
        died_date = '01/01/' + died_date
    if len(married_date) == 4:
        married_date = '01/01/' + married_date
        
    if len(born_date) == 0:
        born_date = None
    if len(died_date) == 0:
        died_date = None
    if len(married_date) == 0:
        married_date = None

    trunc_person = {'name':person['Name'], 'born_date':born_date, 'died_date':died_date, 'married_date':married_date }
    print(trunc_person)
    es.index(index='people', doc_type='person', id=i, body=json.dumps(trunc_person))
    i = i + 1

{'married_date': None, 'died_date': None, 'born_date': None, 'name': 'Ann Waldron'}
{'married_date': '3/4/1848', 'died_date': '6/15/1864', 'born_date': '2/2/1827', 'name': 'Jehiel Armstrong'}
{'married_date': '3/4/1848', 'died_date': '10/16/1888', 'born_date': '1/4/1832', 'name': 'Rebecca Starks'}
{'married_date': None, 'died_date': '01/01/1909', 'born_date': '01/01/1849', 'name': 'Thomas Waldron Armstrong'}
{'married_date': None, 'died_date': '01/01/1930', 'born_date': '9/1/1854', 'name': 'Miles Aldridge Armstrong'}


In [33]:
es.get(index='people', doc_type='person', id=2)

{'_id': '2',
 '_index': 'people',
 '_source': {'born_date': '2/2/1827',
  'died_date': '6/15/1864',
  'married_date': '3/4/1848',
  'name': 'Jehiel Armstrong'},
 '_type': 'person',
 '_version': 1,
 'found': True}

In [142]:
#es.search(index="people", body={"query": {"match": {'name':'Chad David'}}})
#es.search(index="people", body={"query": {"match_all": {}}})

#es.search(index="people", body={"query": {"prefix" : { "name" : "ch" }}})

es.search(index="people", body={"query": {"fuzzy" : { "name" : {"value": "chad", "max_expansions":5}}}})

# NOT WORKS - es.search(index="people", body={"query": {"multi_match": {"fields": ["_all"], "query": "chad", "fuzziness": "AUTO"}}})

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': '30',
    '_index': 'people',
    '_score': 4.061613,
    '_source': {'born_date': '8/15/1971',
     'died_date': None,
     'married_date': '10/27/2008',
     'name': 'Chad David Armstrong'},
    '_type': 'person'},
   {'_id': '338',
    '_index': 'people',
    '_score': 2.688146,
    '_source': {'born_date': '9/6/1921',
     'died_date': '5/8/2007',
     'married_date': '4/23/1948',
     'name': 'Charlotte Evelyn “Char" Chaffee'},
    '_type': 'person'}],
  'max_score': 4.061613,
  'total': 2},
 'timed_out': False,
 'took': 256}

In [4]:
from elasticsearch.client import IndicesClient
#iclient.delete(index='people')

es_index = IndicesClient(es)
es_index.exists(index='people')
#es_index.delete(index='people')


False

In [138]:
body = """{
  "mappings": {
    "person": {
      "properties": {
        "name": {
          "type":   "text"          
        },
        "born_date": {
          "type":   "date",
          "format": "MM/dd/yyyy||yyyy"
        },
        "died_date": {
          "type":   "date",
          "format": "MM/dd/yyyy||yyyy"
        },
        "married_date": {
          "type":   "date",
          "format": "MM/dd/yyyy||yyyy"
        }
      }
    }
  }
}"""

es_index.create(index='people', body=body)

{'acknowledged': True, 'index': 'people', 'shards_acknowledged': True}

In [172]:
import json
messages = []

min_count = 70000
max_count = 100000
count = 0
with open('/Users/chada/Desktop/Inbox/video_messages_2.txt') as json_file:
    for line in json_file:
        #messages.append(json.loads(line))
        if line.strip() != '':
            count += 1
            try:
                if (count > min_count and count < max_count):
                    message = json.loads(line)
                    messages.append(message)                
            except:                
                print(count)
                print('"' + line.strip() + '"')
                
        if count > max_count:
            break
                
print(len(messages)) 
print(messages[0:3])

81838
"{"data": [{"datetime":"2018-03-09 15:32:54"}, ]}"
84533
"{"data": [{"datetime":"2018-03-09 15:31:47"}, ]}"
29997
[{'data': [{'datetime': '2018-03-09 15:16:10'}, {'data': {'error': {'message': 'Black screen in Live stream.', 'err_count': 130349, 'err_subcode': 'BLACK_SCREEN'}, 'video': {'watched_duration': 0, 'video_series_id': '10Rr0YTkbMIPLrtSi6Vn3DWJcAuGrfry', 'video_type': 'live', 'bitrate': 0, 'video_id': 'live'}, 'user': {'user_id': '6CA9574D-13A9-4D13-A63A-D4B5E5BC731E'}}, 'event': 'ERROR_VIDEO_PLAY', 'session_id': 'hetVqrGVJewlpbqW3xGnj41azo1wNf6v'}]}, {'data': [{'datetime': '2018-03-09 15:16:10'}, {'data': {'video': {'watched_duration': 0, 'video_series_id': 'qDZEvujoYGCQkyCk4JOn53yCwSX2arrP', 'video_type': 'live', 'bitrate': 0, 'video_id': 'live'}, 'error': {'message': 'Black screen in Live stream.', 'err_count': 45626, 'err_subcode': 'BLACK_SCREEN'}}, 'event': 'ERROR_VIDEO_PLAY', 'session_id': 'isvweDBnEumVshIpizNGTnihxYD41xgF'}]}, {'data': [{'datetime': '2018-03-09 15

In [173]:
count = 90000
for message in messages:
    #print(message)
    if count % 500 == 0: 
        print(count)
    es.index(index='messages', doc_type='message', id=count, body=json.dumps(message))    
    count += 1

90000
90500
91000
91500
92000
92500
93000
93500
94000
94500
95000
95500
96000
96500
97000
97500
98000
98500
99000
99500
100000
100500
101000
101500
102000
102500
103000
103500
104000
104500
105000
105500
106000
106500
107000
107500
108000
108500
109000
109500
110000
110500
111000
111500
112000
112500
113000
113500
114000
114500
115000
115500
116000
116500
117000
117500
118000
118500
119000
119500


In [38]:
from elasticsearch.client import IndicesClient
#iclient.delete(index='people')

es_index = IndicesClient(es)
#es_index.exists(index='messages')
es_index.delete(index='videos')

{'acknowledged': True}

In [192]:
#localhost

# body = """{
#   "mappings": {
#     "message": {
#       "properties": {
#           "data": {
#               "properties": {
#                   "datetime": {
#                       "type":   "date",
#                       "format": "yyyy-MM-dd HH:mm:ss"
#                   }                  
#               }
#           }
#       }
#     }
#   }  
# }"""

#aws
body = """{
  "mappings": {
    "message": {
      "properties": {
          "insert_date": {
            "type":   "date",
            "format": "yyyy-MM-dd HH:mm:ss"
          },
          "create_date": {
            "type":   "date",
            "format": "yyyy-MM-dd HH:mm:ss"
          }
        }
      }
    }  
}"""

es_index.create(index='messages', body=body)



{'acknowledged': True, 'shards_acknowledged': True}

In [186]:
!pip list


DEPRECATION: The default format will switch to columns in the future. You can use --format=(legacy|columns) (or define a format=(legacy|columns) in your pip.conf under the [list] section) to disable this warning.
alabaster (0.7.7)
anaconda-client (1.4.0)
anaconda-navigator (1.1.0)
appnope (0.1.0)
appscript (1.0.1)
argcomplete (1.0.0)
astropy (1.1.2)
awscli (1.6.2)
Babel (2.2.0)
backports-abc (0.4)
backports.ssl-match-hostname (3.4.0.2)
bcdoc (0.12.2)
beautifulsoup4 (4.4.1)
bitarray (0.8.1)
blaze (0.9.1)
bokeh (0.11.1)
boto (2.33.0)
botocore (0.73.0)
Bottleneck (1.0.0)
cdecimal (2.3)
certifi (2015.4.28)
cffi (1.5.2)
chest (0.2.3)
cloudpickle (0.1.1)
clyent (1.2.1)
colorama (0.2.5)
conda-manager (0.3.1)
configobj (5.0.6)
cryptography (1.3)
cycler (0.10.0)
Cython (0.23.4)
cytoolz (0.7.5)
dask (0.8.1)
datashape (0.5.1)
decorator (4.0.9)
dill (0.2.4)
docutils (0.14)
dynd (0.7.3.dev1)
elasticsearch (6.1.1)
enum34 (1.1.2)
et-xmlfile (1.0.1)
fastcache (1.0.2)
Flask (0.10.1)
Flask-Cors (2.1.2)


In [187]:
!pip show elasticsearch

Name: elasticsearch
Version: 6.1.1
Summary: Python client for Elasticsearch
Home-page: https://github.com/elastic/elasticsearch-py
Author: Honza Král
Author-email: honza.kral@gmail.com
License: Apache License, Version 2.0
Location: /Users/chada/anaconda/envs/gl-env/lib/python2.7/site-packages
Requires: urllib3


In [188]:
!pip show urllib3


Name: urllib3
Version: 1.22
Summary: HTTP library with thread-safe connection pooling, file post, and more.
Home-page: https://urllib3.readthedocs.io/
Author: Andrey Petrov
Author-email: andrey.petrov@shazow.net
License: MIT
Location: /Users/chada/anaconda/envs/gl-env/lib/python2.7/site-packages
Requires: 


In [39]:
body = """{
  "mappings": {
    "video": {
      "properties": {
          "insert_date": {
            "type":   "date",
            "format": "yyyy-MM-dd HH:mm:ss"
          }
        }
      }
    }  
}"""

es_index.create(index='videos', body=body)

{'acknowledged': True, 'shards_acknowledged': True}